# QC Report for Methylation Calling

This notebook generates a comprehensive quality control report for methylation analysis using MethylDackel output data.

In [ ]:
# Import required modules
import warnings

import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
from ugbio_methylation.qc_report_generator import MethylationQCReportGenerator, ReportConfig

# Configure notebook settings
InteractiveShell.ast_node_interactivity = "all"
warnings.filterwarnings("ignore")
pd.set_option("display.max_rows", 10000)
pd.set_option("display.width", 1200)
pd.set_option("display.max_colwidth", 0)

In [ ]:
# Configuration - these parameters are injected during report generation
input_h5_file = ""
input_base_file_name = ""

In [ ]:
# Initialize the report generator
config = ReportConfig(input_h5_file=input_h5_file, input_base_file_name=input_base_file_name)

report_generator = MethylationQCReportGenerator(config)

## Input Parameters

In [ ]:
report_generator.display_sample_info()

## Human Genome: Global Methylation Statistics

In [ ]:
report_generator.generate_global_stats()

## Human Genome: Details of Per-Read Descriptive Statistics

In [ ]:
report_generator.generate_per_read_stats()

## Human Genome: Cytosines in Other Contexts Descriptive Statistics

In [ ]:
report_generator.generate_non_cpg_stats()

## Human Genome: M-bias plots of mean methylation along reads

In [ ]:
report_generator.generate_mbias_analysis()

## Human Genome: M-bias plots of mean methylation on CHH/CHG along reads

In [ ]:
report_generator.generate_mbias_non_cpg_analysis()

## Control Genomes Analysis

In [ ]:
report_generator.generate_control_genome_analysis()

## Human Genome: Additional Details of CpG Methylation and Coverage

In [ ]:
report_generator.generate_histogram_analysis()